In [1]:
# Order preparation for NSE

# STATUS: Completed
# Run-time: 10 seconds

# Dependencies:
# /zdata/nsepkl/*.pkl - for pickles generated by 01_nse_scan program

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 7496, clientId=11) # kavi tws live
# ib = IB().connect('127.0.0.1', 4001, clientId=11) # kavi IBG live
# ib = IB().connect('127.0.0.1', 7499, clientId=11) # kavi tws paper-trade
# ib = IB().connect('127.0.0.1', 4004, clientId=11) # kavi IBG paper-trade

In [2]:
# %%time
import pandas as pd
import numpy as np
import datetime
from os import listdir

#... set limits and penalties
    
m_maxp = 0.015    # % of max margin allowed on net liquidity per scrip to limit positon risk
    
min_rom = 0.5     # 0.5 would be 50% return

base = 0.05       # Upper or Lower base multiple for prices

max_nlvp = 0.8    # max allowable nlv to prevent overall portfolio risk. 0.8 means 80% of NLV.
                  # max available funds for option trades = max_nlvp * NLV - initMargin

#... read the account info
ac = ib.accountSummary()
df_a = util.df(ac)

#... set max margin per position
net_liq = float(df_a[df_a.tag == 'NetLiquidation'].iloc[0].value) 
av_funds = float(df_a[df_a.tag == 'FullAvailableFunds'].iloc[0].value)
max_p = net_liq*m_maxp

#****    PREPARE TO HARVEST   ****
#_________________________________

#... read the positions
ps = ib.portfolio()
df_p = util.df(ps)

df_p['ibSymbol'] = [s.symbol for s in df_p.contract.values]

# get expected price percentage from DTE
def expPricePct(expiry):
    '''Gets expected price percentage from DTE for harvesting trades.
    Assumes max DTE to be 30 days.
    Arg: (expiry) as string 'yyymmdd', e.g. from expPricePct 
    Returns: expected price percentage (xpp) as float
    Ref: http://interactiveds.com.au/software/Linest-poly.xls ... for getting curve function
    '''
#     if dte is to be extracted from contract.lastTradeDateOrContractMonth
    dte = (util.parseIBDatetime(expiry) - datetime.datetime.now().date()).days
    
    if dte > 30:
        dte = 30  # Forces the max DTE to be 30 days
    
    xpp = (103.6008 - 3.63457*dte + 0.03454677*dte*dte)/100
    
    return xpp

# get the harvest as lower of discount from curve * averageCost and discount * marketPrice

expiry = [d.lastTradeDateOrContractMonth for d in df_p.contract]
discount = [m for m in map(expPricePct, expiry)]
df_p['hvstPrice'] = pd.concat([df_p.averageCost*discount, 
                               df_p.marketPrice*(1-np.array(discount))], axis=1).min(axis=1)

df_p.hvstPrice = np.floor(df_p.hvstPrice/base)*base # round down to the nearest 0.05

df_p.loc[df_p.hvstPrice == 0, 'hvstPrice'] = base  # make the 0s to 5 paise

# harvest open positions with hvstPrice
qual_contracts = ib.qualifyContracts(*df_p.contract)
df_p['qual_contracts'] = qual_contracts
df_p['harvestOrder'] = [LimitOrder(action='BUY', totalQuantity=-position, lmtPrice=hvstPrice) for position, hvstPrice in zip(df_p.position, df_p.hvstPrice)]


#****   PREPARE TO SOW !    ****
#________________________________

#... get the lots and margins
# from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

# Rename Symbol and Margin fields
df_paisa = df_paisa.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa = df_paisa.apply(pd.to_numeric, errors='ignore')
df_paisa.marginpct = df_paisa.marginpct.div(100)

# Truncate to 9 characters for ibSymbol
df_paisa['ibSymbol'] = df_paisa.nseSymbol.str.slice(0,9)

# nseSymbol to ibSymbol dictionary for conversion
ntoi = {'M&M': 'MM', 'M&MFIN': 'MM', 'L&TFH': 'LTFH', 'NIFTY': 'NIFTY50'}

# remap ibSymbol, based on the dictionary
df_paisa.ibSymbol = df_paisa.ibSymbol.replace(ntoi)

df_slm = pd.merge(df_p, df_paisa[['ibSymbol', 'Mlot', 'TotMgnPerShr']])

#... make the blacklist

df1 = df_slm.groupby('ibSymbol').sum()
df1['used_margin'] = -df1.position * df1.TotMgnPerShr
df1['max_margin'] = max_p
df1['avail_margin'] = df1.position * df1.TotMgnPerShr + df1.max_margin

df1['max_units'] = (df1.avail_margin/(df1.Mlot*df1.TotMgnPerShr)).apply(np.floor)

blacklist = list(df1[df1.max_units <= 0].index)

#... build the high-pop-roc dataframe

# read pickles from zdata/nse sorted by rom
fs = listdir('./zdata/nsepkl')
dfs = pd.concat([pd.read_pickle(r'./zdata/nsepkl/'+f) for f in fs], axis=0).reset_index(drop=True).sort_values('rom', ascending=False)

# filter out puts which are not in blacklist
dfs_p = dfs[(dfs.right=='P') & (~dfs.ibSymbol.isin(blacklist))]

# filter out puts whose underlying price is greater than minimum rom
df2 = dfs_p[(dfs_p.rom > min_rom)]

# filter puts whose strike price is less than the underlying's mean.
df3 = df2[(df2.strike < df2.undMean)]

In [3]:
# WARNING!!!... Make df the dataframe that you want to execute on!
df = df3.copy()   # make this the last dataframe to get the orders placed

contracts = [c.contract for c in df.ticker]

df.loc[df.expPrice < 0.2, 'expPrice'] = 0.2  # Make the selling price a minimum of 0.2
orders = [LimitOrder(action='SELL', totalQuantity=lot, lmtPrice=expPrice) for lot, expPrice in zip(df.lot, df.expPrice)]

print('{:d} contracts consuming {:,} margin from full available funds of {:,}'.format(len(contracts), sum(df.margin), av_funds))

169 contracts consuming 19,876,719.006999996 margin from full available funds of 17,716,536.54


In [4]:
df_analyze = df[df.ibSymbol.isin(df2.ibSymbol.unique())].groupby(['expiry', 'ibSymbol', 'strike']).apply(max)
df_analyze

conId   ibSymbol    expiry   strike right  \
expiry   ibSymbol  strike                                                   
20190131 ADANIPORT 250.0    339362081  ADANIPORT  20190131    250.0     P   
                   315.0    339362187  ADANIPORT  20190131    315.0     P   
                   320.0    339362192  ADANIPORT  20190131    320.0     P   
                   325.0    339362202  ADANIPORT  20190131    325.0     P   
         ALBK      25.0     339351234       ALBK  20190131     25.0     P   
                   26.0     339351240       ALBK  20190131     26.0     P   
                   27.0     339351251       ALBK  20190131     27.0     P   
                   28.0     339351257       ALBK  20190131     28.0     P   
         APOLLOHOS 1040.0   339328794  APOLLOHOS  20190131   1040.0     P   
         APOLLOTYR 150.0    339343977  APOLLOTYR  20190131    150.0     P   
                   155.0    339343986  APOLLOTYR  20190131    155.0     P   
                   160.0    339343992  APOLLOTYR  20190131    160.0     P   
         ARVIND    20.0     343583133     ARVIND  20190131     20.0     P   
                   30.0     343583140     ARVIND  20190131     30.0     P   
                   40.0     343583150     ARVIND  20190131     40.0     P   
                   50.0     343583157     ARVIND  20190131     50.0     P   
                   60.0     343583167     ARVIND  20190131     60.0     P   
         BANKINDIA 55.0     339329707  BANKINDIA  20190131     55.0     P   
         BIOCON    520.0    339363344     BIOCON  20190131    520.0     P   
                   530.0    339363354     BIOCON  20190131    530.0     P   
                   540.0    339363363     BIOCON  20190131    540.0     P   
                   550.0    339363369     BIOCON  20190131    550.0     P   
         BPCL      190.0    339334787       BPCL  20190131    190.0     P   
                   200.0    339334793       BPCL  20190131    200.0     P   
                   210.0    339334802       BPCL  20190131    210.0     P   
                   220.0    339334808       BPCL  20190131    220.0     P   
                   230.0    339334817       BPCL  20190131    230.0     P   
                   240.0    339334827       BPCL  20190131    240.0     P   
         CONCOR    600.0    339369882     CONCOR  20190131    600.0     P   
         DCBBANK   115.0    339370049    DCBBANK  20190131    115.0     P   
...                               ...        ...       ...      ...   ...   
20190228 MARICO    330.0    343972744     MARICO  20190228    330.0     P   
         MCX       600.0    343977091        MCX  20190228    600.0     P   
         MMFIN     370.0    343960245      MMFIN  20190228    370.0     P   
                   380.0    343960255      MMFIN  20190228    380.0     P   
                   390.0    343960263      MMFIN  20190228    390.0     P   
                   400.0    343960269      MMFIN  20190228    400.0     P   
         MUTHOOTFI 390.0    343998913  MUTHOOTFI  20190228    390.0     P   
                   400.0    343998923  MUTHOOTFI  20190228    400.0     P   
                   410.0    343998928  MUTHOOTFI  20190228    410.0     P   
         NATIONALU 47.5     344000114  NATIONALU  20190228     47.5     P   
                   50.0     344000120  NATIONALU  20190228     50.0     P   
         PFC       72.5     344000330        PFC  20190228     72.5     P   
                   75.0     344000337        PFC  20190228     75.0     P   
         SBIN      245.0    343994338       SBIN  20190228    245.0     P   
                   250.0    343994344       SBIN  20190228    250.0     P   
         SRF       1600.0   343992250        SRF  20190228   1600.0     P   
         TECHM     600.0    343979697      TECHM  20190228    600.0     P   
         TITAN     800.0    343993635      TITAN  20190228    800.0     P   
         TORNTPOWE 205.0    343956325  TORNTPOWE  20190228    205.0     P   
                   210.0    

In [6]:
df_analyze.to_excel('./zdata/analyze.xlsx', index=None, header=True)

# Write to watchlist. This watchlist is to be checked in tws / tradingview for the lowest strike and expiry
# This needs to be imported to IBKR's watchlist
watchlist = [('DES', s, 'STK', 'NSE') for s in df2.ibSymbol.unique()]
df_watch = util.df(watchlist)

df_watch.to_csv('./zdata/watchlist.csv', index=None, header=False)

In [7]:
df_analyze.ibSymbol.unique()

array(['ADANIPORT', 'ALBK', 'APOLLOHOS', 'APOLLOTYR', 'ARVIND',
       'BANKINDIA', 'BIOCON', 'BPCL', 'CONCOR', 'DCBBANK', 'DLF',
       'EQUITAS', 'EXIDEIND', 'FEDERALBN', 'GAIL', 'GMRINFRA', 'GSFC',
       'HCLTECH', 'HDFC', 'HINDALCO', 'HINDPETRO', 'HINDZINC', 'IDBI',
       'IDFC', 'IDFCBANK', 'INDIANB', 'IOC', 'KPIT', 'KSCL', 'MANAPPURA',
       'MMFIN', 'MOTHERSUM', 'MUTHOOTFI', 'ORIENTBAN', 'PFC', 'PVR',
       'TATACHEM', 'TATAPOWER', 'TITAN', 'TORNTPOWE', 'IFCI', 'IGL',
       'JSWSTEEL', 'LT', 'MARICO', 'MCX', 'NATIONALU', 'SBIN', 'SRF',
       'TECHM', 'UPL', 'VOLTAS', 'NIFTY50'], dtype=object)

In [11]:
# Write closed orders placed to file
x  =  r'./zdata/' + datetime.datetime.now().strftime("%Y%m%d_%H%M%S") + "_closing_orders.xlsx"
writer = pd.ExcelWriter(x)
df_p.to_excel(writer, 'all-options', index=False, header=1)
writer.save()

Peer closed connection
